# Basics 2: Listen storage bucket for uploads

In this lesson we modify the previous implementation so that it listens for uploaded files and automatically loads them to database.

## Step 1: Execute the code

In [ ]:
# Imports
import pandas as pd

%run ../../common/jupyter.ipynb
import src_common_database as db
import src_common_storage as st
import src_common_util as util

In [ ]:
# Function for saving sales data to database
def save_sales(file_path):
    print("INFO: Saving " + file_path + " to database")
    
    # Read Sales.csv file from the storage bucket
    bucket = st.create_storage_bucket_client(os.environ['STORAGE_BUCKET'])
    sales_csv = bucket.get_object_contents(file_path)

    # Read Sales.csv data into a Pandas dataframe
    df = pd.read_csv(sales_csv)
    
    # Change dataframe schema to match the database table
    db_df = df.rename(
        columns = {
            'Date': 'date_key',
            'Product': 'product_key',
            'Order': 'order_number',
            'Quantity': 'quantity',
            'Price': 'price',
        },
        inplace = False
    )

    # Generate unique key by concatenating order number and product SKU
    db_df["key"] = db_df["order_number"].astype(str) + "." + db_df["product_key"]

    # Write the data to the "load_sales" view
    database = db.create_engine()
    db_df.to_sql('load_sales', con=database, if_exists='append', index=False)
    print("INFO: Saved " + file_path + " to database successfully")

    # DEBUG: Show the data stored in fact_sales.
    print("DEBUG: Data in database:")
    print(pd.read_sql('select * from fact_sales', con=database))

In [ ]:
# Start listening bucket for uploads
bucket = st.create_storage_bucket_client(os.environ['STORAGE_BUCKET'])
bucket.listen_changes("sales/", ".csv", save_sales)

## Step 2: Add a file to the storage bucket

- Execute `taito open bucket` on command-line to open the storage bucket on your web browser.
- Sign in with access key `minio` and secret key `secret1234`.
- Create a folder named `sales` and upload the **sales-2021-04.csv** file to the folder. If the file already exists, just overwrite it.
- Soon you should see a notification that new data was saved to the database.
- Stop the execution by pressing stop button on the Jupyter Lab web user interface. You can ignore the **KeyboardInterrupt** error message.

## Next lesson: [Basics 3 - Add a new dimension table to database and create data pipeline for it](03.ipynb)